# Orienting Yourself

![The Scientific Python Stack](images/scientific_python_stack.png)
Image: @jakevdp

# How to install packages using conda

If you're using anaconda, you probably already have most (if not all) of these installed. If you installed miniconda:

```
conda install numpy
```

Conda also has channels which allows anybody to distribute their own conda packages. There is an "astropy" channel for AstroPy affiliated packages. You can do:

```
conda install -c astropy astroml
```

To check if a package is available on conda:

```
conda search numpy
```

# How to install packages using pip

Many smaller packages are not available via the `conda` package manager. For these, use `pip`:

```
pip install --no-deps corner
```

## Why prefer conda?

conda is an actual package *manager* that will take care of resolving dependencies optimally.

# NumPy

In [1]:
from __future__ import print_function

import math
import numpy as np

If you use Python for any amount of time, you'll quickly find that there are some things it is not so good at.
In particular, performing repeated operations via loops is one of its weaknesses.

For example, in pure Python:

In [2]:
def add_one(x):
    return [xi + 1 for xi in x]

In [3]:
x = list(range(1000000))
%timeit add_one(x)

10 loops, best of 3: 65 ms per loop


Using numpy we would do:

In [4]:
x = np.arange(1000000)
%timeit np.add(x, 1)

1000 loops, best of 3: 1.23 ms per loop


## Why is pure Python so slow?

![array vs list](images/array_vs_list.png)
Image: @jakevdp


Operations in NumPy are faster than Python functions involving loops, because

- The data type can be checked just once
- The looping then happens in compiled code

## Using NumPy efficiently

The name of the game is moving all array-oriented code into *vectorized* NumPy operations.

In [5]:
# Point coordinates
x = np.random.rand(100000)
y = np.random.rand(100000)

In [7]:
%%timeit
dist = np.empty(len(x))
for i in range(len(x)):
    dist[i] = math.sqrt(x[i]**2 + y[i]**2)

10 loops, best of 3: 84.4 ms per loop


In [9]:
%%timeit
dist = np.sqrt(x**2 + y**2)

1000 loops, best of 3: 514 µs per loop


**Aside:** How many arrays are created in the above cell?

Sometimes you have to get a little creative to "vectorize" things:

In [12]:
x = np.arange(10)**2
x

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [14]:
# difference between adjacent elements
x[1:] - x[:-1]

array([ 1,  3,  5,  7,  9, 11, 13, 15, 17])

In [16]:
# by the way, this is basically the implementation of `np.ediff1d`
np.ediff1d??

### Some interesting properties of numpy functions

Functions that operate element-wise on arrays are known as *universal functions* ("UFuncs").

UFuncs have some methods built-in, which allow for some very interesting, flexible, and fast operations:

In [17]:
x = np.arange(5)
y = np.arange(1, 6)
x + y

array([1, 3, 5, 7, 9])

All operators (like ``+``) actuall call an underlying numpy function: in this case ``np.add``:

In [18]:
np.add(x, y)

array([1, 3, 5, 7, 9])

These ufuncs have some interesting and useful properties:

In [19]:
np.add.accumulate(x)

array([ 0,  1,  3,  6, 10])

In [20]:
np.multiply.accumulate(x)

array([0, 0, 0, 0, 0])

In [21]:
np.multiply.accumulate(y)

array([  1,   2,   6,  24, 120])

In [24]:
x

array([0, 1, 2, 3, 4])

In [28]:
np.maximum.accumulate(x[::-1])

array([4, 4, 4, 4, 4])

In [29]:
np.multiply.identity

1

In [30]:
np.add.outer(x, x)

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])

In [31]:
np.multiply.outer(x, x)

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12],
       [ 0,  4,  8, 12, 16]])

## numpy aggregates

*Aggregates* are functions that take an array and return a smaller-dimension array.

In [32]:
z = np.arange(10, dtype=np.float64).reshape((2, 5))
z

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.]])

In [40]:
np.sum(z, axis=0)

array([  5.,   7.,   9.,  11.,  13.])

In [34]:
# alternate spelling:
z.sum()

45.0

In [36]:
np.mean(z)

4.5

In [38]:
np.min(z), np.max(z)

(0.0, 9.0)

In [44]:
np.sum(z, axis=0)

array([  5.,   7.,   9.,  11.,  13.])

In [41]:
# could also use ufunc 
np.add.reduce(z, axis=0)

array([  5.,   7.,   9.,  11.,  13.])

## Indexing

#### Slice indexing

In [45]:
x = np.arange(15)
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [46]:
x[0:5]

array([0, 1, 2, 3, 4])

In [47]:
x[0:10:2]  # with a stride

array([0, 2, 4, 6, 8])

In [48]:
x[10:0:-2]  # reversed

array([10,  8,  6,  4,  2])

This sort of indexing does not make a copy:

In [49]:
y = x[0:10:2]
y[0] = 100.  # modify y
y

array([100,   2,   4,   6,   8])

In [50]:
# x is modified:
x

array([100,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14])

#### Indexing with indicies

In [51]:
x = np.arange(15)
y = x[[1, 2, 4]]
y

array([1, 2, 4])

In [52]:
y[0] = 100
y

array([100,   2,   4])

In [53]:
# x is not modified
x

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

#### Indexing with booleans

In [54]:
x = np.arange(5)
x

array([0, 1, 2, 3, 4])

In [55]:
mask = np.array([True, True, False, True, False])
x[mask]

array([0, 1, 3])

In [56]:
# creates a copy
y = x[mask]
y[0] = 100.
print(y)
print(x)

[100   1   3]
[0 1 2 3 4]


*How do you remember which type of indexing creates a copy?*

NumPy arrays are stored as a chunk of data and a set of strides in each dimension.
Boolean and arbitrary indicies cannot be represented this way, so numpy must make a copy.

#### More on masking

All indexing operations also work in assigning to an array. Here we demonstrate assignment with booleans.

For example, imagine you have an array of data where negative values indicate some kind of error.

In [57]:
x = np.array([1, 2, 3, -999, 2, 4, -999])

How might you clean this array, setting all negative values to, say, zero?

In [58]:
for i in range(len(x)):
    if x[i] < 0:
        x[i] = 0
x

array([1, 2, 3, 0, 2, 4, 0])

In [59]:
x = np.array([1, 2, 3, -999, 2, 4, -999])

mask = (x < 0)
mask

array([False, False, False,  True, False, False,  True], dtype=bool)

And the mask can be used directly to set the value you desire:

In [60]:
x[mask] = 0
x

array([1, 2, 3, 0, 2, 4, 0])

Often you'll see this done in a separate step:

In [62]:
x = np.array([1, 2, 3, -999, 2, 4, -999])
x[x < 0] = 0
x

array([1, 2, 3, 0, 2, 4, 0])

In [68]:
# additional boolean operations: invert
x = np.array([1, 2, 3, -999, 2, 4, -999])
x[~(x < 0)] = 0
x


array([   0,    0,    0, -999,    0,    0, -999])

In [71]:
x = np.array([1, 2, 3, -999, 2, 4, -999])
x[(x < 0) | (x > 3)] = 0
x

array([1, 2, 3, 0, 2, 0, 0])

## Broadcasting

In [72]:
x = np.arange(4)
x

array([0, 1, 2, 3])

In [73]:
x + 3

array([3, 4, 5, 6])

![broadcasting 1d](images/broadcast_1D.png)

In [74]:
x = np.array([[0,  0,  0],
              [10, 10, 10],
              [20, 20, 20],
              [30, 30, 30]])
y = np.array([0, 1, 2])
print("x shape:", x.shape)
print("y shape:   ", y.shape)

x shape: (4, 3)
y shape:    (3,)


In [75]:
# If x and y are different dimensions, shape is padded on left with 1s
# before broadcasting.
x + y

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

![numpy broadcasting](images/numpy_broadcasting.svg)

In [76]:
x = np.array([[0],
              [10],
              [20],
              [30]])
y = np.array([0, 1, 2])
print("x shape:", x.shape)
print("y shape:   ", y.shape)

x shape: (4, 1)
y shape:    (3,)


In [81]:
x + y

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

**Broadcasting rules:**

1. If the two arrays differ in their number of dimensions, the shape of the array with fewer dimensions is *padded* with ones on its leading (left) side.

2. If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is *stretched* to match the other shape.

3. If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

## Mini exercises

Assume you have $N$ points in $D$ dimensions, represented by an array of shape `(N, D)`, where there are some mising values scattered throughout the points.

1. Count the number of points (rows) with no missing values, using `np.any` or `np.all`.

2. Clean the array of the points with missing values.

3. Construct the matrix ``M``, the centered and normalized version of the ``X`` array: $$ M_{ij} = (X_{ij} - \mu_j) / \sigma_j $$ using `np.mean` and `np.std`.  This is one version of *whitening* the array.

In [83]:
np.random.seed(0)
X = np.random.rand(5000)
X[np.random.randint(0, 5000, size=500)] = np.nan  # ~10% missing
X = X.reshape((1000, 5))  # 1000 points in 5 dimensions

In [112]:
# 1. Compute the number of points (rows) with no missing values, using `np.any` or `np.all`.
mask = np.all(~np.isnan(X), axis=1)
mask.sum()

612

In [96]:
# 2. Clean the array, leaving only rows with no missing values
Y = X[mask]
Y.shape

(612, 5)

In [99]:
# 3. Compute the whitened version of the array using np.mean and np.std.
(Y - np.mean(Y, axis=0)) / np.std(Y, axis=0)

array([[ 1.06529173,  0.09242815,  0.28895283,  1.37495433, -1.51438865],
       [-1.41794214, -1.64994524,  1.22435157,  0.88599369,  1.20944498],
       [ 0.53028299, -1.22817041,  1.62049802,  0.03598805, -0.34291462],
       ..., 
       [ 1.53325011,  0.58772122, -0.90299997,  1.0223077 , -0.32292258],
       [-1.03929087, -0.96220483, -0.88700535, -1.18773479,  0.60570199],
       [-0.95616059, -0.91269895, -1.02149246, -0.37245591,  1.51143122]])

## What else is in NumPy?

- `numpy.random`: Random number generation
- `numpy.linalg`: Some linear algebra routines
- `numpy.fft`: Fast Fourier Transform

In [ ]:
print(np.random.__doc__)

# SciPy

Interestingly, scipy predates numpy by more than half a decade (circa 1999), even though it is built on top of numpy.

Originally "scipack", a collection of wrappers for Fortran NetLib libraries.

In [101]:
# contents of scipy:
import scipy
print(scipy.__doc__)


SciPy: A scientific computing package for Python

Documentation is available in the docstrings and
online at http://docs.scipy.org.

Contents
--------
SciPy imports all the functions from the NumPy namespace, and in
addition provides:

Subpackages
-----------
Using any of these subpackages requires an explicit import.  For example,
``import scipy.cluster``.

::

 cluster                      --- Vector Quantization / Kmeans
 fftpack                      --- Discrete Fourier Transform algorithms
 integrate                    --- Integration routines
 interpolate                  --- Interpolation Tools
 io                           --- Data input and output
 linalg                       --- Linear algebra routines
 linalg.blas                  --- Wrappers to BLAS library
 linalg.lapack                --- Wrappers to LAPACK library
 misc                         --- Various utilities that don't have
                                  another home.
 ndimage                      --- n-dime

Note the overlap:

- `numpy.fft` / `scipy.fft`
- `numpy.linalg` / `scipy.linalg`

Why the duplication? The scipy routines are based on Fortran libraries, whereas numpy is C-only.

# AstroPy

Project started in 2011, in response to increasing duplication in Python astronomy ecosystem.

Initially brought together several existing Python packages:

- `astropy.io.fits` (formerly `pyfits`)
- `astropy.io.ascii` (formerly `asciitable`)
- `astropy.wcs` (formerly `pywcs`)
- `astropy.cosmology` (formerly `cosmolopy`)

Now also contains:

- `astropy.table` (Table class and table operations)
- `astropy.units` (Quantity: arrays with units)
- `astropy.coordinates` (astronomical coordinate systems)
- `astropy.time` (UTC, UT, MJD, etc)
- `astropy.stats` (additional stats not in scipy)
- `astropy.modeling` (simple model fitting)
- `astropy.vo` (virtual observatory)
- `astropy.io.votable`
- `astropy.analytic_functions`

## Example: Coordinates

In [102]:
from astropy import coordinates as coords
from astropy import units as u

/home/kyle/.conda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [103]:
ra = 360. * np.random.rand(100)
dec = -90. + 180. * np.random.rand(100)
print("RA:", ra[:5], "...")
print("Dec:", dec[:5], "...")

RA: [  22.26271521  183.12057268  245.43571113  135.14011884   46.60902808] ...
Dec: [  0.84336721  71.28940527  67.98133383 -57.76496669  62.82170868] ...


In [104]:
c = coords.SkyCoord(ra, dec, unit=(u.deg, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    [(22.26271521, 0.84336721), (183.12057268, 71.28940527),
     (245.43571113, 67.98133383), (135.14011884, -57.76496669),
     (46.60902808, 62.82170868), (5.5463205, 1.75300981),
     (139.22724099, -70.89876917), (267.38584817, -33.93293061),
     (356.59862418, 33.6502332), (166.70614453, 70.81810387),
     (106.17018389, -70.4970478), (124.80006278, 19.73757575),
     (33.09518851, 32.06320665), (162.04076361, -40.28813887),
     (225.7663043, 46.19530728), (224.48786841, -71.40090351),
     (4.5968499, -46.85816015), (228.14350723, -89.91162652),
     (239.64711432, 0.49282502), (180.72883022, -42.58152806),
     (340.15654779, -75.94785495), (92.8961079, 68.55866164),
     (306.68972711, -54.24030273), (62.71976297, 12.11756194),
     (94.30386281, -41.13622255), (73.76355903, 17.54992362),
     (318.71501869, 27.77339407), (285.66924461, 40.60643253),
     (162.17415904, -36.76930584), (136.08830588, -81.49407869),
     (4.64446431, 64.30889

In [105]:
# convert to galactic
g = c.galactic
g

<SkyCoord (Galactic): (l, b) in deg
    [(142.37028781, -60.60101337), (127.37230118, 45.50149802),
     (100.49884144, 38.71850453), (275.87053715, -7.5951703),
     (137.6665351, 3.87813007), (108.06497461, -60.27162952),
     (287.10403859, -14.9623307), (356.15234471, -3.29831988),
     (107.72578741, -27.29362883), (134.52292868, 43.88003306),
     (281.46855763, -24.37217664), (203.80955011, 27.83935091),
     (142.27463499, -27.75021987), (278.84553583, 16.74828645),
     (78.02777848, 57.8109045), (312.74161612, -10.9804884),
     (319.01590877, -69.22446783), (302.98924546, -27.05610163),
     (10.25291144, 37.82692152), (293.49026089, 19.39890292),
     (312.59812132, -38.62184719), (145.71418151, 21.60079545),
     (343.95068476, -35.46134659), (180.53815112, -27.72519608),
     (248.59233783, -23.55871339), (182.99565049, -15.97021682),
     (75.16627147, -14.40484285), (71.10929272, 15.2873244),
     (277.16460109, 19.88635456), (295.2489923, -22.26134368),
     (119.37897

In [106]:
# access longitude or latitude
g.l

<Longitude [ 142.37028781, 127.37230118, 100.49884144, 275.87053715,
             137.6665351 , 108.06497461, 287.10403859, 356.15234471,
             107.72578741, 134.52292868, 281.46855763, 203.80955011,
             142.27463499, 278.84553583,  78.02777848, 312.74161612,
             319.01590877, 302.98924546,  10.25291144, 293.49026089,
             312.59812132, 145.71418151, 343.95068476, 180.53815112,
             248.59233783, 182.99565049,  75.16627147,  71.10929272,
             277.16460109, 295.2489923 , 119.37897502, 117.42508474,
             114.18005303, 319.43162298, 159.8258784 , 211.69193909,
             208.87177497, 212.17417162, 198.19156249,  62.30981967,
              51.79074719,  33.01879165, 310.73787241, 106.57051623,
             174.05492536, 122.63526923,  86.03553116,  38.92746275,
             109.06000508, 199.49015343,   8.93071087, 274.15184992,
             308.86627869,  93.42409172,  86.83008336, 251.30642995,
             103.13637159,  68.122

In [107]:
type(g.l)

astropy.coordinates.angles.Longitude

In [108]:
# get underlying numpy array
g.l.degree

array([ 142.37028781,  127.37230118,  100.49884144,  275.87053715,
        137.6665351 ,  108.06497461,  287.10403859,  356.15234471,
        107.72578741,  134.52292868,  281.46855763,  203.80955011,
        142.27463499,  278.84553583,   78.02777848,  312.74161612,
        319.01590877,  302.98924546,   10.25291144,  293.49026089,
        312.59812132,  145.71418151,  343.95068476,  180.53815112,
        248.59233783,  182.99565049,   75.16627147,   71.10929272,
        277.16460109,  295.2489923 ,  119.37897502,  117.42508474,
        114.18005303,  319.43162298,  159.8258784 ,  211.69193909,
        208.87177497,  212.17417162,  198.19156249,   62.30981967,
         51.79074719,   33.01879165,  310.73787241,  106.57051623,
        174.05492536,  122.63526923,   86.03553116,   38.92746275,
        109.06000508,  199.49015343,    8.93071087,  274.15184992,
        308.86627869,   93.42409172,   86.83008336,  251.30642995,
        103.13637159,   68.12213634,   74.63146094,  145.56809

# Other astro packages

[AstroPy-affiliated packages](http://www.astropy.org/affiliated/index.html)

[PyPI packages](https://pypi.python.org/pypi?:action=browse&show=all&c=385&c=387) with topic "Astronomy"